# LCRTest1

## Setup and Data

In [1]:
# Imports functions into the program
import os
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#These are imported specifically for the use of the histogram.
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

#Imported for categorization of the Sources
from fractional_variability import fvar
from fermi_catalog import select_bll, select_fsrq, select_bcu

In [2]:
table = "new_db_Feb2023.csv"
opened = open(table,"r")
readed = pd.read_csv(table, sep=",", na_filter=True)
readed = readed.fillna(-3333)

cadence = 'weekly'
cadence_df = readed.loc[(readed['cadence'] == cadence) #& \
                        # (readed['ts2'] <= 4.) & \
                        # (readed['photon_flux_error2'] <  readed['photon_flux2'])
                        ]
cadence_df.loc[cadence_df['ts2'] <= 9., 'photon_flux2'] = -3333
cadence_df.loc[cadence_df['photon_flux_error2'] < cadence_df['photon_flux2'], 'photon_flux2'] = -3333
print(cadence_df)

TSTART = np.amin(cadence_df['tmin'])
TSTOP = np.amax(cadence_df['tmax'])

names = np.unique(readed['source_name']) #.drop_duplicates(subset=['source_name'], inplace=False)#[1]

         Unnamed: 0        source_name cadence       tmin       tmax  \
3050           3050  4FGL J0001.2-0747  weekly  239587201  240192001   
3051           3051  4FGL J0001.5+2113  weekly  239587201  240192001   
3052           3052  4FGL J0003.3-1928  weekly  239587201  240192001   
3053           3053  4FGL J0004.3+4614  weekly  239587201  240192001   
3054           3054  4FGL J0004.4-4737  weekly  239587201  240192001   
...             ...                ...     ...        ...        ...   
4132745     4132745  4FGL J2358.0-4601  weekly  698630401  699235201   
4132746     4132746  4FGL J2358.3-1021  weekly  698630401  699235201   
4132747     4132747  4FGL J2358.3+3830  weekly  698630401  699235201   
4132748     4132748  4FGL J2359.0+3922  weekly  698630401  699235201   
4132749     4132749  4FGL J2359.2-3134  weekly  698630401  699235201   

         photon_flux2  photon_flux_error2  photon_index2  return_code2  \
3050          -3333.0       -3.333000e+03       -3333.00     

In [3]:
fsrq_df = select_fsrq()
fsrq_names = fsrq_df['Source_Name']

fsrq_fmatrix = []
fsrq_fractionalflux = []
fsrq_nanbins = []
fsrq_nanbinsmatrix = []
for i, fsrq in enumerate(fsrq_names):
    # print(i, fsrq)
    source_array = cadence_df.loc[cadence_df['source_name'] == fsrq]
    averageflux = np.mean(source_array[source_array['photon_flux2']!=-3333]['photon_flux2'])
    
    flux = np.array(source_array[source_array['photon_flux2']!=-3333]['photon_flux2'])
    flux_err = np.array(source_array[source_array['photon_flux2']!=-3333]['photon_flux_error2'])
    var, var_err = fvar(flux, flux_err)

    factors = source_array['photon_flux2']/averageflux
    fsrq_nanbins.append(len(source_array[source_array['photon_flux2']== -3333]))
    fsrq_nanbinsmatrix.append(np.array(source_array['photon_flux2'] == -3333))
    fsrq_fmatrix.append(factors.tolist())
    fsrq_fractionalflux.append([fsrq, averageflux, var, var_err])

fsrq_fmatrix = np.array(fsrq_fmatrix)
fsrq_nanbinsmatrix = np.array(fsrq_nanbinsmatrix)

##


-----------------------------
Selecting FSRQ type blazars
Total number of variable objects: 572


In [ ]:
# We want to express each source as a point on a plot of Duration vs Average Flux
# To do this, we want to note the total duration in the flaring state
# I'll write a function that takes an array of blazars, using the functions Michel made, and calculates the average flux of those points.
# Each point noted down will be stored in an array, along with the average flux of each state.

In [7]:
pd.DataFrame(fsrq_fmatrix)

,0,1,2,3,4,5,6,7,8,9,...,750,751,752,753,754,755,756,757,758,759
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-4.547067e+11,-4.547067e+11,-4.547067e+11,-4.547067e+11,-4.547067e+11,-4.547067e+11,-4.547067e+11,-4.547067e+11,-4.547067e+11,-4.547067e+11,...,-4.547067e+11,-4.547067e+11,-4.547067e+11,-4.547067e+11,-4.547067e+11,-4.547067e+11,-4.547067e+11,-4.547067e+11,-4.547067e+11,-4.547067e+11
2,-1.116895e+11,-1.116895e+11,-1.116895e+11,-1.116895e+11,-1.116895e+11,-1.116895e+11,-1.116895e+11,-1.116895e+11,-1.116895e+11,-1.116895e+11,...,-1.116895e+11,-1.116895e+11,-1.116895e+11,-1.116895e+11,-1.116895e+11,-1.116895e+11,-1.116895e+11,-1.116895e+11,-1.116895e+11,-1.116895e+11
3,-5.983842e+11,-5.983842e+11,-5.983842e+11,-5.983842e+11,-5.983842e+11,-5.983842e+11,-5.983842e+11,-5.983842e+11,-5.983842e+11,-5.983842e+11,...,-5.983842e+11,-5.983842e+11,-5.983842e+11,-5.983842e+11,-5.983842e+11,-5.983842e+11,-5.983842e+11,-5.983842e+11,-5.983842e+11,-5.983842e+11
4,-4.974627e+11,-4.974627e+11,-4.974627e+11,-4.974627e+11,-4.974627e+11,-4.974627e+11,-4.974627e+11,-4.974627e+11,-4.974627e+11,-4.974627e+11,...,-4.974627e+11,-4.974627e+11,-4.974627e+11,-4.974627e+11,-4.974627e+11,-4.974627e+11,-4.974627e+11,-4.974627e+11,-4.974627e+11,-4.974627e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567,-7.158505e+10,-7.158505e+10,-7.158505e+10,-7.158505e+10,-7.158505e+10,-7.158505e+10,-7.158505e+10,-7.158505e+10,-7.158505e+10,-7.158505e+10,...,-7.158505e+10,-7.158505e+10,-7.158505e+10,-7.158505e+10,-7.158505e+10,-7.158505e+10,-7.158505e+10,-7.158505e+10,-7.158505e+10,-7.158505e+10
568,-3.560897e+08,-3.560897e+08,-3.560897e+08,-3.560897e+08,-3.560897e+08,-3.560897e+08,-3.560897e+08,-3.560897e+08,-3.560897e+08,-3.560897e+08,...,-3.560897e+08,-3.560897e+08,-3.560897e+08,-3.560897e+08,-3.560897e+08,-3.560897e+08,-3.560897e+08,-3.560897e+08,-3.560897e+08,-3.560897e+08
569,-5.481908e+11,-5.481908e+11,-5.481908e+11,-5.481908e+11,-5.481908e+11,-5.481908e+11,-5.481908e+11,-5.481908e+11,-5.481908e+11,-5.481908e+11,...,-5.481908e+11,-5.481908e+11,-5.481908e+11,-5.481908e+11,-5.481908e+11,-5.481908e+11,-5.481908e+11,-5.481908e+11,-5.481908e+11,-5.481908e+11
570,-9.657604e+10,-9.657604e+10,-9.657604e+10,-9.657604e+10,-9.657604e+10,-9.657604e+10,-9.657604e+10,-9.657604e+10,-9.657604e+10,-9.657604e+10,...,-9.657604e+10,-9.657604e+10,-9.657604e+10,-9.657604e+10,-9.657604e+10,-9.657604e+10,-9.657604e+10,-9.657604e+10,-9.657604e+10,-9.657604e+10


In [8]:
fmatrix = []
fractionalflux = []
for i, name in enumerate(names):
    print(i, name)
    source_array = cadence_df.loc[cadence_df['source_name'] == name]
    averageflux = np.nanmean(source_array['photon_flux2'])
    flux = np.array(source_array['photon_flux2'])
    flux_err = np.array(source_array['photon_flux_error2'])

    factors = flux/averageflux
    fmatrix.append(factors.tolist())
    fractionalflux.append([name,factors,averageflux,flux,flux_err])
# pd.DataFrame(fractionalflux)

fmatrix = np.array(fmatrix)
print(fmatrix.shape)

0 4FGL J0001.2-0747
1 4FGL J0001.5+2113
2 4FGL J0003.3-1928
3 4FGL J0004.3+4614
4 4FGL J0004.4-4001
5 4FGL J0004.4-4737
6 4FGL J0005.9+3824
7 4FGL J0007.7+4008
8 4FGL J0009.1-5012
9 4FGL J0009.3+5030
10 4FGL J0010.6+2043
11 4FGL J0010.6-3025
12 4FGL J0011.4+0057
13 4FGL J0014.1+1910
14 4FGL J0014.3-0500
15 4FGL J0014.9+3212
16 4FGL J0016.2-0016
17 4FGL J0017.0-0649
18 4FGL J0017.5-0514
19 4FGL J0019.2-5640
20 4FGL J0019.6+7327
21 4FGL J0021.5-2552
22 4FGL J0021.9-5140
23 4FGL J0022.5+0608
24 4FGL J0023.7+4457
25 4FGL J0023.7-6820
26 4FGL J0024.4+4647
27 4FGL J0024.7+0349
28 4FGL J0025.7-4801
29 4FGL J0028.4+2001
30 4FGL J0029.0-7044
31 4FGL J0030.2-1647
32 4FGL J0030.3-4224
33 4FGL J0030.6-0212
34 4FGL J0032.3-5522
35 4FGL J0033.5-1921
36 4FGL J0033.9+3858
37 4FGL J0034.0-4116
38 4FGL J0035.2+1514
39 4FGL J0035.8+6131
40 4FGL J0035.9+5950
41 4FGL J0036.9+1832
42 4FGL J0037.8+1239
43 4FGL J0038.2-2459
44 4FGL J0039.1+4330
45 4FGL J0043.8+3425
46 4FGL J0044.2-8424
47 4FGL J0045.1-3706
48

In [10]:
pd.DataFrame(fractionalflux)

,0,1,2,3,4
0,4FGL J0001.2-0747,"[1.0270270270270907, 1.0270270270270907, 1.027...",-3245.289474,"[-3333.0, -3333.0, -3333.0, -3333.0, -3333.0, ...","[-3333.0, 1.42e-08, -3333.0, 5.13e-09, 4.15e-0..."
1,4FGL J0001.5+2113,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",-3333.000000,"[-3333.0, -3333.0, -3333.0, -3333.0, -3333.0, ...","[1.55e-08, 1.97e-08, -3333.0, 3.02e-08, -3333...."
2,4FGL J0003.3-1928,"[1.003963011889046, 1.003963011889046, 1.00396...",-3319.843421,"[-3333.0, -3333.0, -3333.0, -3333.0, -3333.0, ...","[-3333.0, -3333.0, 1.92e-09, -3333.0, -3333.0,..."
3,4FGL J0004.3+4614,"[1.0013175230566564, 1.0013175230566564, 1.001...",-3328.614474,"[-3333.0, -3333.0, -3333.0, -3333.0, -3333.0, ...","[-3333.0, -3333.0, -3333.0, -3333.0, 5.32e-08,..."
4,4FGL J0004.4-4001,"[1.0052910052910067, 1.0052910052910067, 1.005...",-3315.457895,"[-3333.0, -3333.0, -3333.0, -3333.0, -3333.0, ...","[-3333.0, -3333.0, 3.01e-08, 2.06e-08, -3333.0..."
...,...,...,...,...,...
1520,4FGL J2358.0-4601,"[1.0052910052910486, 1.0052910052910486, 1.005...",-3315.457895,"[-3333.0, -3333.0, -3333.0, -3333.0, -3333.0, ...","[-3333.0, -3333.0, -3333.0, 3.27e-08, -3333.0,..."
1521,4FGL J2358.3+3830,"[1.0160427807487014, 1.0160427807487014, 1.016...",-3280.373684,"[-3333.0, -3333.0, -3333.0, -3333.0, -3333.0, ...","[-3333.0, 1.29e-08, -3333.0, -3333.0, -3333.0,..."
1522,4FGL J2358.3-1021,"[1.005291005291015, 1.005291005291015, 1.00529...",-3315.457895,"[-3333.0, -3333.0, -3333.0, -3333.0, -3333.0, ...","[2.82e-08, -3333.0, -3333.0, 1.79e-08, -3333.0..."
1523,4FGL J2359.0+3922,"[1.0079575596817805, 1.0079575596817805, 1.007...",-3306.686842,"[-3333.0, -3333.0, -3333.0, -3333.0, -3333.0, ...","[1.97e-08, -3333.0, 3.92e-09, -3333.0, -3333.0..."


In [11]:
def Esras(source_array):
    fmatrix = []
    fractionalflux = []
    for i, name in enumerate(names):
        print(i, name)
        source_array = cadence_df.loc[cadence_df['source_name'] == name]
        averageflux = np.nanmean(source_array['photon_flux2'])
        flux = np.array(source_array['photon_flux2'])
        flux_err = np.array(source_array['photon_flux_error2'])

        factors = flux/averageflux
        fmatrix.append(factors.tolist())
        fractionalflux.append([name,factors,averageflux,flux,flux_err])
    # pd.DataFrame(fractionalflux)

    fmatrix = np.array(fmatrix)
    print(fmatrix.shape)
    return fmatrix

In [14]:
cadence_df
source_array

,Unnamed: 0,source_name,cadence,tmin,tmax,photon_flux2,photon_flux_error2,photon_index2,return_code2,ts2
4574,4574,4FGL J2359.2-3134,weekly,239587201,240192001,-3333.0,-3.333000e+03,-3333.00,0.0,0.55
9149,9149,4FGL J2359.2-3134,weekly,240192001,240796801,-3333.0,-3.333000e+03,-3333.00,102.0,-0.00
15249,15249,4FGL J2359.2-3134,weekly,240796801,241401601,-3333.0,3.180000e-08,8.09,102.0,9.92
19824,19824,4FGL J2359.2-3134,weekly,241401601,242006401,-3333.0,-3.333000e+03,-3333.00,0.0,0.19
25924,25924,4FGL J2359.2-3134,weekly,242006401,242611201,-3333.0,-3.333000e+03,-3333.00,102.0,-0.00
...,...,...,...,...,...,...,...,...,...,...
4111399,4111399,4FGL J2359.2-3134,weekly,696211201,696816001,-3333.0,-3.333000e+03,-3333.00,0.0,0.13
4115974,4115974,4FGL J2359.2-3134,weekly,696816001,697420801,-3333.0,-3.333000e+03,-3333.00,102.0,-0.00
4120549,4120549,4FGL J2359.2-3134,weekly,697420801,698025601,-3333.0,2.090000e-08,2.93,0.0,1.06
4128174,4128174,4FGL J2359.2-3134,weekly,698025601,698630401,-3333.0,-3.333000e+03,-3333.00,102.0,-0.00


In [15]:
fsrq_fmatrix

array([[            nan,             nan,             nan, ...,
                    nan,             nan,             nan],
       [-4.54706685e+11, -4.54706685e+11, -4.54706685e+11, ...,
        -4.54706685e+11, -4.54706685e+11, -4.54706685e+11],
       [-1.11689472e+11, -1.11689472e+11, -1.11689472e+11, ...,
        -1.11689472e+11, -1.11689472e+11, -1.11689472e+11],
       ...,
       [-5.48190789e+11, -5.48190789e+11, -5.48190789e+11, ...,
        -5.48190789e+11, -5.48190789e+11, -5.48190789e+11],
       [-9.65760371e+10, -9.65760371e+10, -9.65760371e+10, ...,
        -9.65760371e+10, -9.65760371e+10, -9.65760371e+10],
       [            nan,             nan,             nan, ...,
                    nan,             nan,             nan]])